# Day 6: Resonant Collinearity

## Import libraries

In [68]:
import copy

## Import data

In [ ]:
# *** [IMPORT DATA] ***
# NOTE: In the given puzzle input:
# - xxx
# =====================================================================================================================
# ! Open the file for reading mode (= default mode if the mode is not specified)
file = open("../data/24_day-8_input-test.txt", "r") 

# Read all the data in the file
file_data = file.read().strip()

# Separate data by line to create rows for grid
grid = file_data.split("\n")

# Separate data in EACH row to represent EACH column
for i in range(len(grid)):
    grid[i] = list(grid[i])

print(grid)
# ====================================================================================================================

## Helper functions

## Part 1

In [ ]:
# *** [PART 1] ***
# ! PROBLEM: xxx
# - TODO: xxx
# ====================================================================================================================
# ! Create a deep (independent) copy of the grid data, such that changes made to the copy do not affect the original grid to still test/re-run Part 1 with the correct INITIAL (and not modified) grid
# - NOTE: Not using a deep copy will modify the original grid after running Part 1, therefore no correct output will be calculated anymore
part1_grid = copy.deepcopy(grid)


# ====================================================================================================================

## Part 2

In [ ]:
# *** [PART 2] ***
# ! PROBLEM: xxx
# - TODO: xxx
#====================================================================================================================
# ! Create a deep (independent) copy of the grid data, such that changes made to the copy do not affect the original grid to still test/re-run Part 1 with the correct INITIAL (and not modified) grid
# - NOTE: Not using a deep copy will modify the original grid after running Part 1, therefore no correct output will be calculated anymore
part2_grid = copy.deepcopy(grid)
